In [1]:
# Data processing
import pandas as pd
import numpy as np
# Standardize the data
from sklearn.preprocessing import StandardScaler
# Modeling 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval
from sklearn.decomposition import PCA

In [6]:
train_dt = pd.read_csv("/home/wt/meta_target/data/train_dtV2.csv")
dt = pd.read_csv("/home/data/sdc/wt/model_data/ml_input_pca.csv")

In [20]:
dt

,cell,is_dep,gene,exp_PC1,exp_PC2,exp_PC3,exp_PC4,exp_PC5,exp_PC6,exp_PC7,...,gene_PC91,gene_PC92,gene_PC93,gene_PC94,gene_PC95,gene_PC96,gene_PC97,gene_PC98,gene_PC99,gene_PC100
0,ACH-000001,0,FASN,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,...,-0.783433,1.222521,-0.117844,-1.180916,0.489429,6.411101,-1.291333,4.338066,2.154690,-3.947985
1,ACH-000001,0,CPT1A,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,...,1.760688,-0.050529,0.594583,2.253033,-2.895312,0.495416,0.155392,0.052814,-0.629862,-3.223677
2,ACH-000001,1,DHFR2,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,...,0.089213,-0.418685,-0.178663,-0.314786,0.006747,0.339003,0.081292,0.849139,-0.344751,-0.226163
3,ACH-000001,0,"AMT,DLD,GCSH,GLDC",-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,...,6.657687,2.175304,-2.715057,14.802428,13.435182,11.940548,-2.445253,4.747263,8.620059,-5.038838
4,ACH-000001,0,"HADHA,HADHB,ACAA2",-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,...,1.522276,11.036809,-3.356520,3.489670,-4.820659,4.365528,-2.837405,9.476923,-12.899784,13.714948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307161,ACH-002847,0,SLC44A3,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,...,-0.061162,-0.243907,-0.078555,0.335004,-0.193368,0.236844,-0.539125,-1.251575,-0.564338,0.520884
307162,ACH-002847,0,PFKFB4,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,...,0.765647,-0.236150,-1.151003,0.688749,1.130870,0.931269,-1.331601,-1.194248,-0.519116,2.173915
307163,ACH-002847,0,RFK,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,...,0.914174,0.679717,-0.040071,0.201663,-0.056712,0.933769,0.689966,0.384210,-0.350915,-1.329807
307164,ACH-002847,0,FDFT1,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,...,-1.970303,-1.544308,-3.034350,-2.282498,2.652457,2.227629,0.505568,0.819886,1.882958,1.782551


In [12]:
train_dt

,id,is_dep,cell
0,ENSG00000169710,0,ACH-000001
1,ENSG00000110090,0,ACH-000001
2,ENSG00000178700,1,ACH-000001
3,ENSG00000091140 and ENSG00000140905 and ENSG00...,0,ACH-000001
4,ENSG00000084754 and ENSG00000138029 and ENSG00...,0,ACH-000001
...,...,...,...
307161,ENSG00000143036,0,ACH-002847
307162,ENSG00000114268,0,ACH-002847
307163,ENSG00000135002,0,ACH-002847
307164,ENSG00000079459,0,ACH-002847


In [2]:
cell_exp = pd.read_csv("/home/data/sdb/wt/model_data/cell_gene_exp_vs_normal_filter.csv")
gene_pc = pd.read_csv("/home/wt/meta_target/data/gene_pca.csv")

In [9]:
cell_exp

,cell,TSPAN6,TNMD,FGR,CFH,FUCA2,GCLC,NIPAL3,ENPP4,SEMA3F,...,CCL3,RDM1,SSTR3,ZNF229,PAGR1,TMEM265,IQCJ-SCHIP1,FAM95C,POLR2J3,CDR1
0,ACH-000001,0.834052,0.000000,0.018975,0.117574,1.020171,1.267026,1.160431,1.277001,1.752333,...,0.119034,35.613526,0.061974,0.659649,0.409680,0.627333,8.429846,0.030733,2.151418,0.169902
1,ACH-000002,1.567636,0.000000,0.455358,0.283170,1.229716,2.146425,1.715752,0.703614,0.819235,...,0.161383,6.364074,0.000000,22.136255,0.811025,0.339882,18.323165,6.934119,2.702116,0.000000
2,ACH-000003,1.550883,0.244251,0.000000,0.012161,1.440753,1.017566,0.624509,0.370511,0.646105,...,0.156191,61.865601,0.501594,0.007053,0.584730,0.805583,1.308347,0.235235,3.247730,0.000000
3,ACH-000004,19.349066,0.000000,0.097581,8.588410,1.174782,2.545963,2.785231,2.094138,4.121307,...,0.000000,18.165942,0.000000,7.129488,0.862093,0.104526,12.282771,0.017557,4.582243,0.000000
4,ACH-000005,21.795952,0.000000,0.022169,9.974503,1.234576,2.887259,2.706622,2.646389,5.394369,...,0.010199,26.896015,0.014678,8.341004,0.909194,0.046796,16.634064,0.017557,4.008452,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,ACH-002664,1.295513,0.000000,0.014310,0.539113,1.168902,1.025357,1.037424,0.710221,0.913804,...,0.000000,51.495409,0.000000,0.000000,0.585808,1.000111,1.258608,0.373564,2.836891,0.000000
1000,ACH-002669,0.908638,0.000000,0.004817,0.778894,1.470751,0.960387,1.033912,0.609453,0.443109,...,1.320381,51.368699,0.501594,0.007053,0.383519,0.944348,1.397429,1.093248,3.126475,0.000000
1001,ACH-002672,1.491753,0.000000,0.000000,2.808837,1.326717,1.636174,2.312882,1.327402,2.113979,...,0.000000,103.475898,0.067533,2.725855,0.770438,1.034909,18.959074,0.369933,4.716915,0.000000
1002,ACH-002680,1.459109,0.754776,0.381291,1.437083,2.142221,1.263061,0.764964,0.825137,2.388434,...,0.000000,2.306139,0.067910,0.000000,0.596285,0.003632,1.968299,0.000000,5.750088,0.203779


In [3]:
gene_feat = pd.read_csv("/home/wt/meta_target/data/all_train_gene_cpg.csv")

In [11]:
gene_feat

,id,fea-1,fea-2,fea-3,fea-4,fea-5,fea-6,fea-7,fea-8,fea-9,...,fea-3238,fea-3239,fea-3240,fea-3241,fea-3242,fea-3243,fea-3244,fea-3245,fea-3246,fea-3247
0,ENSG00000169710,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ENSG00000110090,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ENSG00000178700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSG00000091140 and ENSG00000140905 and ENSG00...,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,ENSG00000084754 and ENSG00000138029 and ENSG00...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,ENSG00000021461,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
924,ENSG00000184999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
925,ENSG00000243480,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
926,ENSG00000132958,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
sample_info = pd.read_csv("/home/data/sdc/wt/model_data/new_model/cell_net_filter_exp/raw/train_cell_info.csv")

In [14]:
sample_info

,cell,cell_index
0,ACH-000001,0
1,ACH-000004,1
2,ACH-000005,2
3,ACH-000007,3
4,ACH-000009,4
...,...,...
679,ACH-002659,679
680,ACH-002664,680
681,ACH-002669,681
682,ACH-002672,682


In [15]:
X_train = dt.drop(columns=["is_dep","cell","gene"])
y_train = dt.is_dep

In [16]:
X_train

,exp_PC1,exp_PC2,exp_PC3,exp_PC4,exp_PC5,exp_PC6,exp_PC7,exp_PC8,exp_PC9,exp_PC10,...,gene_PC91,gene_PC92,gene_PC93,gene_PC94,gene_PC95,gene_PC96,gene_PC97,gene_PC98,gene_PC99,gene_PC100
0,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,-9.321838,0.222364,-8.586790,...,-0.783433,1.222521,-0.117844,-1.180916,0.489429,6.411101,-1.291333,4.338066,2.154690,-3.947985
1,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,-9.321838,0.222364,-8.586790,...,1.760688,-0.050529,0.594583,2.253033,-2.895312,0.495416,0.155392,0.052814,-0.629862,-3.223677
2,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,-9.321838,0.222364,-8.586790,...,0.089213,-0.418685,-0.178663,-0.314786,0.006747,0.339003,0.081292,0.849139,-0.344751,-0.226163
3,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,-9.321838,0.222364,-8.586790,...,6.657687,2.175304,-2.715057,14.802428,13.435182,11.940548,-2.445253,4.747263,8.620059,-5.038838
4,-15.194231,9.955007,-10.610324,11.348358,-10.901722,11.252601,16.191492,-9.321838,0.222364,-8.586790,...,1.522276,11.036809,-3.356520,3.489670,-4.820659,4.365528,-2.837405,9.476923,-12.899784,13.714948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307161,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,-7.256014,-0.551323,1.506344,...,-0.061162,-0.243907,-0.078555,0.335004,-0.193368,0.236844,-0.539125,-1.251575,-0.564338,0.520884
307162,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,-7.256014,-0.551323,1.506344,...,0.765647,-0.236150,-1.151003,0.688749,1.130870,0.931269,-1.331601,-1.194248,-0.519116,2.173915
307163,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,-7.256014,-0.551323,1.506344,...,0.914174,0.679717,-0.040071,0.201663,-0.056712,0.933769,0.689966,0.384210,-0.350915,-1.329807
307164,-17.502485,-0.648967,2.773653,-8.584688,18.318570,-21.489684,5.828374,-7.256014,-0.551323,1.506344,...,-1.970303,-1.544308,-3.034350,-2.282498,2.652457,2.227629,0.505568,0.819886,1.882958,1.782551


In [8]:
# Initiate scaler
sc = StandardScaler()
# Standardize the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train),index=X_train.index, columns=X_train.columns)

In [9]:
###逻辑回归
clf = SGDClassifier(loss="log_loss", penalty="elasticnet", fit_intercept=True, early_stopping=True,n_iter_no_change=100)

# use a full grid over all parameters
param_grid = {
    'max_iter': [500,1000,2000,3000,4000,5000],
    'learning_rate': ['optimal', 'invscaling', 'adaptive']
}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, n_jobs = 30, scoring="f1")

In [14]:
grid_search.fit(X_train_transformed, y_train)

/home/wt/miniconda3/envs/dl/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/home/wt/miniconda3/envs/dl/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/wt/miniconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File "/home/wt/miniconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 149, in

GridSearchCV(estimator=SGDClassifier(early_stopping=True, loss='log_loss',
                                     n_iter_no_change=100,
                                     penalty='elasticnet'),
             n_jobs=30,
             param_grid={'learning_rate': ['optimal', 'invscaling', 'adaptive'],
                         'max_iter': [500, 1000, 2000, 3000, 4000, 5000]},
             scoring='f1')

In [ ]:
grid_search.best_params_

{'learning_rate': 'optimal', 'max_iter': 1000}

In [ ]:
####CV
from sklearn.model_selection import KFold
splits = KFold(n_splits=10,shuffle=True,random_state=2023052701)

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(sample_info)))):
    print('Fold {}'.format(fold + 1))
    ###get fold train and test data
    X_train_idx = train_dt.cell.isin(sample_info.cell[train_idx])
    X_train = train_dt[X_train_idx]
    X_test_idx = train_dt.cell.isin(sample_info.cell[val_idx])
    X_test = train_dt[X_test_idx]
    ###get fold train and test data
    fold_train_exp_idx = cell_exp.cell.isin(sample_info.cell[train_idx])
    fold_train_exp = cell_exp[fold_train_exp_idx].reset_index()
    pca_exp = PCA(n_components=100)
    fold_train_exp_pca = pd.DataFrame(pca_exp.fit_transform(fold_train_exp.iloc[:,2:7995]), 
                                      columns=["exp_pca-" + str(i) for i in range(100)])
    fold_train_exp_pca["cell"] = fold_train_exp.cell
    
    fold_train_gene_idx = gene_feat.id.isin(X_train.id)
    fold_train_gene_feat = gene_feat[fold_train_gene_idx].reset_index()
    pca_gene = PCA(n_components=100)
    fold_train_gene_pca = pd.DataFrame(pca_gene.fit_transform(fold_train_gene_feat.iloc[:,2:3249]),
                                       columns=["gene_pca-" + str(i) for i in range(100)])
    fold_train_gene_pca["id"] = fold_train_gene_feat.id
    
    X_train_dt = X_train.merge(fold_train_exp_pca, on='cell', how='left').merge(fold_train_gene_pca, on='id', how='left').drop(columns=["id","cell","is_dep"])
    y_train = X_train.is_dep
    
    fold_test_exp_idx = cell_exp.cell.isin(sample_info.cell[val_idx])
    fold_test_exp = cell_exp[fold_test_exp_idx].reset_index()
    fold_test_exp_pca = pd.DataFrame(pca_exp.transform(fold_test_exp.iloc[:,2:7995]), 
                                     columns=["exp_pca-" + str(i) for i in range(100)])
    fold_test_exp_pca["cell"] = fold_test_exp.cell
    
    fold_test_gene_idx = gene_feat.id.isin(X_test.id)
    fold_test_gene_feat = gene_feat[fold_test_gene_idx].reset_index()
    fold_test_gene_pca = pd.DataFrame(pca_gene.transform(fold_test_gene_feat.iloc[:,2:3249]), 
                                      columns=["gene_pca-" + str(i) for i in range(100)])
    fold_test_gene_pca["id"] = fold_test_gene_feat.id
    
    X_test_dt = X_test.merge(fold_test_exp_pca, on='cell', how='left').merge(fold_test_gene_pca, on='id', how='left').drop(columns=["id","cell","is_dep"])
    y_test = np.array(X_test.is_dep)
    y_test_gene = np.array(X_test.id)
    
    # Initiate scaler
    sc = StandardScaler()
    # Standardize the training dataset
    X_train_transformed = pd.DataFrame(sc.fit_transform(X_train_dt),index=X_train_dt.index, columns=X_train_dt.columns)
    X_test_transformed = pd.DataFrame(sc.transform(X_test_dt),index=X_test_dt.index, columns=X_test_dt.columns)
    ###fit model
    clf = SGDClassifier(loss="log_loss", penalty="elasticnet", fit_intercept=True,
                        learning_rate="optimal", max_iter=1000, n_iter_no_change=100, early_stopping=True)
    clf.fit(X_train_transformed, y_train)
    y_pred = clf.predict(X_test_transformed)
    y_pred_raw = clf.predict_proba(X_test_transformed)
    y_pred_raw = y_pred_raw[:,1]
    res = pd.DataFrame({"preds":y_pred,"preds_raw":y_pred_raw,"label":y_test,"genes":y_test_gene})
    res.to_csv("~/meta_target/data/cv/ml_logistic/fold_"+str(fold)+".csv")

In [15]:
##SVM
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline

# feature_map_nystroem = Nystroem(random_state=2023061001)
linear_svc = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True, n_iter_no_change=100, early_stopping=True)
# nystroem_approx_svm = Pipeline(
#     [("feature_map", feature_map_nystroem), 
#      ("svm", linear_svc)]
# )

grid_param = {
    # "feature_map__gamma": [0.1,1,10],
    # "feature_map__n_components": [100,200,500,1000],
    "max_iter": [500,1000,2000,3000,4000,5000],
    "learning_rate": ['optimal', 'invscaling', 'adaptive']
}

In [ ]:
gridsearch = GridSearchCV(linear_svc, param_grid=grid_param, n_jobs = 30, scoring="f1") # Fit grid search
gridsearch.fit(X_train_transformed, y_train)

In [ ]:
gridsearch.best_params_

In [8]:
####CV
from sklearn.model_selection import KFold
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline

splits = KFold(n_splits=10,shuffle=True,random_state=2023052701)

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(sample_info)))):
    print('Fold {}'.format(fold + 1))
    ###get fold train and test data
    fold_train_exp_idx = cell_exp.cell.isin(sample_info.cell[train_idx])
    fold_train_exp = cell_exp[fold_train_exp_idx].reset_index()
    pca = PCA(n_components=100)
    fold_train_exp_pca = pd.DataFrame(pca.fit_transform(fold_train_exp.iloc[:,2:7995]), columns=["pca-" + str(i) for i in range(100)])
    fold_train_exp_pca["cell"] = fold_train_exp.cell
    
    X_train_idx = train_dt.cell.isin(sample_info.cell[train_idx])
    X_train = train_dt[X_train_idx]
    X_train_dt = X_train.merge(fold_train_exp_pca, on='cell', how='left').merge(gene_pc, on='id', how='left').drop(columns=["id","cell","is_dep"])
    y_train = X_train.is_dep
    
    fold_test_exp_idx = cell_exp.cell.isin(sample_info.cell[val_idx])
    fold_test_exp = cell_exp[fold_test_exp_idx].reset_index()
    fold_test_exp_pca = pd.DataFrame(pca.transform(fold_test_exp.iloc[:,2:7995]), columns=["pca-" + str(i) for i in range(100)])
    fold_test_exp_pca["cell"] = fold_test_exp.cell
    
    X_test_idx = train_dt.cell.isin(sample_info.cell[val_idx])
    X_test = train_dt[X_test_idx]
    X_test_dt = X_test.merge(fold_test_exp_pca, on='cell', how='left').merge(gene_pc, on='id', how='left').drop(columns=["id","cell","is_dep"])
    y_test = np.array(X_test.is_dep)
    y_test_gene = np.array(X_test.id)
    
    # Initiate scaler
    sc = StandardScaler()
    # Standardize the training dataset
    X_train_transformed = pd.DataFrame(sc.fit_transform(X_train_dt),index=X_train_dt.index, columns=X_train_dt.columns)
    X_test_transformed = pd.DataFrame(sc.transform(X_test_dt),index=X_test_dt.index, columns=X_test_dt.columns)
    ###fit model
    #feature_map_nystroem = Nystroem(random_state=2023061001, gamma=0.1, n_components=1000)
    linear_svc = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True, n_iter_no_change=100, 
                               early_stopping=True,
                               learning_rate="optimal",max_iter=4000)
    # nystroem_approx_svm = Pipeline(
    #     [("feature_map", feature_map_nystroem), 
    #      ("svm", linear_svc)]
    # )
    linear_svc.fit(X_train_transformed, y_train)
    y_pred = linear_svc.predict(X_test_transformed)
    res = pd.DataFrame({"preds":y_pred,"label":y_test,"genes":y_test_gene})
    res.to_csv("/home/wt/meta_target/data/cv/ml_svm/fold_"+str(fold)+".csv")

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
Fold 10


In [17]:
###losso regression on full dataset 
train_dt = train_dt.merge(cell_exp, on='cell', how='left').merge(gene_feat, on='id', how='left')

In [21]:
X_train = train_dt.drop(columns=["is_dep","cell","id"])
y_train = train_dt.is_dep

In [22]:
X_train

,TSPAN6,TNMD,FGR,CFH,FUCA2,GCLC,NIPAL3,ENPP4,SEMA3F,CFTR,...,fea-3238,fea-3239,fea-3240,fea-3241,fea-3242,fea-3243,fea-3244,fea-3245,fea-3246,fea-3247
0,0.834052,0.0,0.018975,0.117574,1.020171,1.267026,1.160431,1.277001,1.752333,6.218532,...,0,0,0,0,0,0,0,0,0,0
1,0.834052,0.0,0.018975,0.117574,1.020171,1.267026,1.160431,1.277001,1.752333,6.218532,...,0,0,0,0,0,0,0,0,0,0
2,0.834052,0.0,0.018975,0.117574,1.020171,1.267026,1.160431,1.277001,1.752333,6.218532,...,0,0,0,0,0,0,0,0,0,0
3,0.834052,0.0,0.018975,0.117574,1.020171,1.267026,1.160431,1.277001,1.752333,6.218532,...,0,0,0,0,0,0,0,0,0,0
4,0.834052,0.0,0.018975,0.117574,1.020171,1.267026,1.160431,1.277001,1.752333,6.218532,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307161,0.881089,0.0,0.000000,0.592531,1.547507,1.315322,1.233883,0.955115,0.289880,0.009658,...,0,0,0,0,0,0,0,0,0,0
307162,0.881089,0.0,0.000000,0.592531,1.547507,1.315322,1.233883,0.955115,0.289880,0.009658,...,0,0,0,0,0,0,0,0,0,0
307163,0.881089,0.0,0.000000,0.592531,1.547507,1.315322,1.233883,0.955115,0.289880,0.009658,...,0,0,0,0,0,0,0,0,0,0
307164,0.881089,0.0,0.000000,0.592531,1.547507,1.315322,1.233883,0.955115,0.289880,0.009658,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Initiate scaler
sc = StandardScaler()
# Standardize the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train),index=X_train.index, columns=X_train.columns)

In [ ]:
clf = SGDClassifier(loss="log_loss", penalty="l1", fit_intercept=True, early_stopping=True,n_iter_no_change=100)
# use a full grid over all parameters
param_grid = {
    'max_iter': [500,1000,2000],
    'learning_rate': ['optimal', 'invscaling', 'adaptive']
}
# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, n_jobs = 30, scoring="f1")
grid_search.fit(X_train_transformed, y_train)